In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [21]:
# These are how we are going to transform the images
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [30]:
# load my datasets
train_dataset = datasets.ImageFolder(root='train', transform=transform)
test_dataset = datasets.ImageFolder(root='test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [31]:
# Create the model to process images
class EmotionClassifier(nn.Module):
    def __init__(self):
        super(EmotionClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 7)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = x.view(-1, 128 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = EmotionClassifier().to(device)

In [32]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [33]:
# Train model
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

Epoch 5/50, Loss: 1.5736
Epoch 6/50, Loss: 1.5433
Epoch 7/50, Loss: 1.5227
Epoch 8/50, Loss: 1.4928
Epoch 9/50, Loss: 1.4741
Epoch 10/50, Loss: 1.4489
Epoch 11/50, Loss: 1.4280
Epoch 12/50, Loss: 1.4077
Epoch 13/50, Loss: 1.3890
Epoch 14/50, Loss: 1.3681
Epoch 15/50, Loss: 1.3565
Epoch 16/50, Loss: 1.3394
Epoch 17/50, Loss: 1.3253
Epoch 18/50, Loss: 1.3115
Epoch 19/50, Loss: 1.2977
Epoch 20/50, Loss: 1.2832
Epoch 21/50, Loss: 1.2693
Epoch 22/50, Loss: 1.2564
Epoch 23/50, Loss: 1.2456
Epoch 24/50, Loss: 1.2292
Epoch 25/50, Loss: 1.2144
Epoch 26/50, Loss: 1.2050
Epoch 27/50, Loss: 1.1977
Epoch 28/50, Loss: 1.1830
Epoch 29/50, Loss: 1.1676
Epoch 30/50, Loss: 1.1561
Epoch 31/50, Loss: 1.1441
Epoch 32/50, Loss: 1.1305
Epoch 33/50, Loss: 1.1220
Epoch 34/50, Loss: 1.1098
Epoch 35/50, Loss: 1.1004
Epoch 36/50, Loss: 1.0854
Epoch 37/50, Loss: 1.0744
Epoch 38/50, Loss: 1.0665
Epoch 39/50, Loss: 1.0535
Epoch 40/50, Loss: 1.0368
Epoch 41/50, Loss: 1.0270
Epoch 42/50, Loss: 1.0166
Epoch 43/50, Loss

In [34]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')

Accuracy on test set: 56.97%


In [ ]:
# Save it
torch.save(model.state_dict(), 'emotion_model.pth')